In [2]:
import pandas as pd
import os
from IPython.display import clear_output
from functools import reduce

In [3]:
BATCH_SIZE = 100000
FILES_LIST = ["/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ac_1..515.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ab_1..534.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ad_1..547.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ae_1..524.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.ab_1..2641.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.ac_aa_1..536.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.ac_ab_1..532.g.vcf.gz",
             "/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.ac_ac_1..521.g.vcf.gz",]


BLANK_CELL_PREFIX = "\./\."
BLANK_VAL = "."

In [3]:
# df = pd.read_csv("/sci/archive/michall/superk/ukbb_data/batch_sample_LDL_ids.aa_ac_1..515.g.vcf.gz",compression='gzip',skiprows=3381,nrows=100000,sep="\t")


In [5]:
threshold = 0.10
count = 0
# pd.read_csv('test.csv', sep='|', skiprows=range(1, 10))
# skiprows=count*BATCH_SIZE + 3381
while True:
    clear_output(wait=True)
    print(count)
    dfs = []
#     df = pd.read_csv("/sci/archive/michall/roeizucker/col_hackaton/batch_sample_LDL_ids.aa_ad_547.g.vcf.gz",compression='gzip',skiprows=list(range(3381)) + list(range(3382,count*BATCH_SIZE + 3382)),nrows=BATCH_SIZE,sep="\t")
    for file in FILES_LIST:
        dfs.append(pd.read_csv(file,compression='gzip',skiprows=list(range(3381)) + list(range(3382,count*BATCH_SIZE + 3382)),nrows=BATCH_SIZE,sep="\t"))
    joined_df = reduce(lambda x,y: pd.merge(x,y, on=["#CHROM","POS"], how='inner'), dfs).replace(f'^{BLANK_CELL_PREFIX}.*$', BLANK_VAL, regex=True)
    if count > 100:
        break

# Count the occurrences of "." in each row

    dot_counts = joined_df.eq(BLANK_VAL).sum(axis=1)
    rows_to_remove = dot_counts > (joined_df.shape[1] * threshold)
    filtered_df = joined_df[~rows_to_remove].replace(BLANK_VAL, ".")
    filtered_df.to_csv(f"/sci/archive/michall/roeizucker/col_hackaton/joined_multiple_files/joined_{count}.csv")
    count += 1


7


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.